# Part III: Core FastAPI Concepts

## Chapter 8: Response Handling

Response handling is a critical aspect of API development. The way you structure, optimize, and deliver responses directly impacts your API's performance, usability, and developer experience. This chapter explores advanced response handling techniques, from status codes and performance optimization to streaming and custom response formats.

---

### 8.1 Status Codes: Setting Explicit Status Codes for Different Scenarios

HTTP status codes communicate the outcome of a request. Using the correct status code is essential for building intuitive, RESTful APIs that clients can handle properly.

#### Understanding HTTP Status Code Categories

| Category | Range | Meaning |
|----------|-------|---------|
| **1xx** | 100-199 | Informational - Request received, continuing process |
| **2xx** | 200-299 | Success - Request successfully received, understood, accepted |
| **3xx** | 300-399 | Redirection - Further action needed to complete request |
| **4xx** | 400-499 | Client Error - Request contains bad syntax or cannot be fulfilled |
| **5xx** | 500-599 | Server Error - Server failed to fulfill a valid request |

#### Common Status Codes for APIs

```python
from fastapi import FastAPI, status

app = FastAPI()


@app.get("/status-demo")
async def status_demo():
    """
    Common HTTP status codes for APIs.
    """
    return {
        "2xx Success": {
            200: "OK - Standard success response",
            201: "Created - Resource successfully created",
            202: "Accepted - Request accepted for processing",
            204: "No Content - Success with no response body",
            206: "Partial Content - Partial resource returned",
        },
        "3xx Redirection": {
            301: "Moved Permanently - Resource moved to new URL",
            302: "Found - Resource temporarily at different URL",
            304: "Not Modified - Resource unchanged since last request",
            307: "Temporary Redirect - Temporary redirect with same method",
            308: "Permanent Redirect - Permanent redirect with same method",
        },
        "4xx Client Errors": {
            400: "Bad Request - Malformed request syntax",
            401: "Unauthorized - Authentication required",
            403: "Forbidden - Authentication succeeded but no permission",
            404: "Not Found - Resource does not exist",
            405: "Method Not Allowed - HTTP method not supported",
            409: "Conflict - Request conflicts with current state",
            422: "Unprocessable Entity - Validation error",
            429: "Too Many Requests - Rate limit exceeded",
        },
        "5xx Server Errors": {
            500: "Internal Server Error - Generic server error",
            501: "Not Implemented - Server doesn't support functionality",
            502: "Bad Gateway - Invalid response from upstream server",
            503: "Service Unavailable - Server temporarily unavailable",
            504: "Gateway Timeout - Upstream server timeout",
        },
    }
```

#### Setting Status Codes

**Method 1: Using the `status_code` Parameter**

```python
from fastapi import FastAPI, status
from pydantic import BaseModel

app = FastAPI()


class Item(BaseModel):
    id: int
    name: str


@app.post("/items", status_code=status.HTTP_201_CREATED)
async def create_item(item: Item):
    """
    Create a new item. Returns 201 Created.
    """
    return {"id": item.id, "name": item.name}


@app.delete("/items/{item_id}", status_code=status.HTTP_204_NO_CONTENT)
async def delete_item(item_id: int):
    """
    Delete an item. Returns 204 No Content.
    """
    # Perform deletion
    return None  # 204 responses should have no body
```

**Method 2: Using the `Response` Object**

```python
from fastapi import FastAPI, Response, status
from pydantic import BaseModel

app = FastAPI()


class User(BaseModel):
    id: int
    username: str


@app.post("/users")
async def create_user(user: User, response: Response):
    """
    Create a user and set the status code dynamically.
    """
    # Simulate checking if user already exists
    existing_user = False
    
    if existing_user:
        response.status_code = status.HTTP_409_CONFLICT
        return {"error": "User already exists"}
    
    response.status_code = status.HTTP_201_CREATED
    response.headers["Location"] = f"/users/{user.id}"
    
    return {"id": user.id, "username": user.username}
```

**Method 3: Using Response Classes**

```python
from fastapi import FastAPI
from fastapi.responses import JSONResponse
from pydantic import BaseModel

app = FastAPI()


class ErrorResponse(BaseModel):
    code: str
    message: str


@app.post("/items")
async def create_item(name: str):
    """
    Create an item with explicit JSONResponse.
    """
    if not name:
        return JSONResponse(
            status_code=400,
            content={"code": "INVALID_INPUT", "message": "Name is required"},
        )
    
    return JSONResponse(
        status_code=201,
        content={"id": 1, "name": name},
        headers={"Location": "/items/1"},
    )
```

#### Status Codes by Resource Action

```python
from typing import Annotated
from fastapi import FastAPI, HTTPException, status, Path
from pydantic import BaseModel

app = FastAPI()

# Simulated database
items_db = {
    1: {"id": 1, "name": "Laptop", "price": 999.99},
    2: {"id": 2, "name": "Mouse", "price": 29.99},
}


class Item(BaseModel):
    name: str
    price: float


class ItemResponse(BaseModel):
    id: int
    name: str
    price: float


# CREATE - POST
@app.post("/items", status_code=status.HTTP_201_CREATED, response_model=ItemResponse)
async def create_item(item: Item):
    """
    Create a new resource.
    
    Status Codes:
    - 201 Created: Resource successfully created
    - 400 Bad Request: Invalid input data
    - 409 Conflict: Resource already exists
    """
    new_id = max(items_db.keys()) + 1
    new_item = {"id": new_id, **item.model_dump()}
    items_db[new_id] = new_item
    return new_item


# READ - GET (Single)
@app.get("/items/{item_id}", response_model=ItemResponse)
async def get_item(item_id: Annotated[int, Path(ge=1)]):
    """
    Get a single resource.
    
    Status Codes:
    - 200 OK: Resource found and returned
    - 404 Not Found: Resource doesn't exist
    """
    item = items_db.get(item_id)
    if item is None:
        raise HTTPException(
            status_code=status.HTTP_404_NOT_FOUND,
            detail=f"Item {item_id} not found",
        )
    return item


# READ - GET (List)
@app.get("/items", status_code=status.HTTP_200_OK)
async def list_items(skip: int = 0, limit: int = 10):
    """
    List resources.
    
    Status Codes:
    - 200 OK: List returned successfully
    - 206 Partial Content: Partial list returned (for pagination)
    """
    items = list(items_db.values())[skip : skip + limit]
    return {"items": items, "count": len(items), "total": len(items_db)}


# UPDATE - PUT (Full Replace)
@app.put("/items/{item_id}", response_model=ItemResponse)
async def replace_item(item_id: int, item: Item):
    """
    Replace an entire resource.
    
    Status Codes:
    - 200 OK: Resource updated, return updated resource
    - 201 Created: New resource created (if upsert)
    - 204 No Content: Resource updated, no body returned
    - 400 Bad Request: Invalid input
    - 404 Not Found: Resource doesn't exist
    """
    if item_id not in items_db:
        raise HTTPException(
            status_code=status.HTTP_404_NOT_FOUND,
            detail=f"Item {item_id} not found",
        )
    
    updated_item = {"id": item_id, **item.model_dump()}
    items_db[item_id] = updated_item
    return updated_item


# UPDATE - PATCH (Partial Update)
@app.patch("/items/{item_id}", response_model=ItemResponse)
async def update_item(item_id: int, item: dict):
    """
    Partially update a resource.
    
    Status Codes:
    - 200 OK: Resource updated
    - 204 No Content: Resource updated, no body
    - 400 Bad Request: Invalid input
    - 404 Not Found: Resource doesn't exist
    - 409 Conflict: Update conflicts with current state
    """
    if item_id not in items_db:
        raise HTTPException(
            status_code=status.HTTP_404_NOT_FOUND,
            detail=f"Item {item_id} not found",
        )
    
    existing = items_db[item_id]
    for key, value in item.items():
        if key in existing and key != "id":
            existing[key] = value
    
    return existing


# DELETE
@app.delete("/items/{item_id}", status_code=status.HTTP_204_NO_CONTENT)
async def delete_item(item_id: int):
    """
    Delete a resource.
    
    Status Codes:
    - 204 No Content: Resource deleted successfully
    - 202 Accepted: Deletion accepted, processing
    - 404 Not Found: Resource doesn't exist
    """
    if item_id not in items_db:
        raise HTTPException(
            status_code=status.HTTP_404_NOT_FOUND,
            detail=f"Item {item_id} not found",
        )
    
    del items_db[item_id]
    return None
```

#### Conditional Responses with Status Codes

```python
from fastapi import FastAPI, Request, HTTPException, status
from fastapi.responses import JSONResponse
import hashlib

app = FastAPI()

# Simulated resource
resource = {
    "id": 1,
    "content": "Hello, World!",
    "etag": "abc123",
    "last_modified": "Mon, 15 Jan 2024 12:00:00 GMT",
}


@app.get("/conditional")
async def conditional_get(request: Request):
    """
    Conditional GET using If-None-Match and If-Modified-Since.
    Returns 304 Not Modified if conditions are met.
    """
    # Check If-None-Match (ETag)
    if_none_match = request.headers.get("if-none-match")
    if if_none_match and if_none_match == resource["etag"]:
        return JSONResponse(status_code=status.HTTP_304_NOT_MODIFIED)
    
    # Check If-Modified-Since
    if_modified_since = request.headers.get("if-modified-since")
    if if_modified_since and if_modified_since == resource["last_modified"]:
        return JSONResponse(status_code=status.HTTP_304_NOT_MODIFIED)
    
    return JSONResponse(
        content=resource,
        headers={
            "ETag": resource["etag"],
            "Last-Modified": resource["last_modified"],
        },
    )


@app.put("/conditional")
async def conditional_put(request: Request, content: str):
    """
    Conditional PUT using If-Match.
    Returns 412 Precondition Failed if conditions are not met.
    """
    # Check If-Match (ETag)
    if_match = request.headers.get("if-match")
    if if_match and if_match != resource["etag"]:
        return JSONResponse(
            status_code=status.HTTP_412_PRECONDITION_FAILED,
            content={"error": "Resource has been modified"},
        )
    
    # Update resource
    new_etag = hashlib.md5(content.encode()).hexdigest()
    resource["content"] = content
    resource["etag"] = new_etag
    
    return JSONResponse(
        content=resource,
        headers={"ETag": new_etag},
    )
```

#### Status Codes for Error Handling

```python
from fastapi import FastAPI, HTTPException, status
from fastapi.responses import JSONResponse
from pydantic import BaseModel

app = FastAPI()


class ErrorResponse(BaseModel):
    code: str
    message: str
    details: dict | None = None


# Custom exception for consistent error responses
class APIError(Exception):
    def __init__(self, status_code: int, code: str, message: str, details: dict | None = None):
        self.status_code = status_code
        self.code = code
        self.message = message
        self.details = details


@app.exception_handler(APIError)
async def api_error_handler(request, exc: APIError):
    """Handler for custom API exceptions."""
    return JSONResponse(
        status_code=exc.status_code,
        content={
            "code": exc.code,
            "message": exc.message,
            "details": exc.details,
        },
    )


@app.get("/items/{item_id}")
async def get_item(item_id: int):
    """
    Get item with custom error responses.
    """
    if item_id <= 0:
        raise APIError(
            status_code=status.HTTP_400_BAD_REQUEST,
            code="INVALID_ID",
            message="Item ID must be positive",
            details={"provided_id": item_id},
        )
    
    if item_id > 100:
        raise APIError(
            status_code=status.HTTP_404_NOT_FOUND,
            code="NOT_FOUND",
            message=f"Item {item_id} does not exist",
        )
    
    if item_id == 50:
        raise APIError(
            status_code=status.HTTP_403_FORBIDDEN,
            code="ACCESS_DENIED",
            message="You don't have permission to access this item",
        )
    
    return {"id": item_id, "name": f"Item {item_id}"}
```

---

### 8.2 `JSONResponse` and `ORJSONResponse`: Optimizing Response Performance

FastAPI's default response class is `JSONResponse`, which uses Python's built-in `json` module. For better performance, especially with large payloads, you can use `ORJSONResponse` which uses the fast `orjson` library.

#### Understanding JSON Response Performance

| Library | Speed | Features |
|---------|-------|----------|
| `json` (stdlib) | Baseline | Built-in, no dependencies |
| `ujson` | 2-3x faster | C library, some edge case differences |
| `orjson` | 3-5x faster | Rust library, datetime support, strict spec |
| `msgspec` | 5-10x faster | Rust library, schema validation |

#### Using JSONResponse

```python
from fastapi import FastAPI
from fastapi.responses import JSONResponse
from pydantic import BaseModel

app = FastAPI()


class User(BaseModel):
    id: int
    name: str
    email: str


@app.get("/json-response")
async def get_json():
    """
    Standard JSONResponse using Python's json module.
    """
    return JSONResponse(
        content={
            "message": "Hello from JSONResponse",
            "data": [1, 2, 3, 4, 5],
        },
        status_code=200,
        headers={"X-Custom": "value"},
    )


@app.get("/json-with-model")
async def json_with_model():
    """
    JSONResponse with Pydantic model serialization.
    """
    user = User(id=1, name="Alice", email="alice@example.com")
    
    return JSONResponse(
        content=user.model_dump(),
        status_code=200,
    )
```

#### Installing and Using ORJSONResponse

First, install `orjson`:

```bash
# Using uv
uv add orjson

# Using pip
pip install orjson
```

```python
from fastapi import FastAPI
from fastapi.responses import ORJSONResponse
from pydantic import BaseModel
from datetime import datetime
from uuid import uuid4

app = FastAPI()


class User(BaseModel):
    id: int
    name: str
    email: str
    created_at: datetime
    uuid: str


@app.get("/orjson-response", response_class=ORJSONResponse)
async def get_orjson():
    """
    ORJSONResponse is 3-5x faster than JSONResponse.
    
    Note: response_class=ORJSONResponse sets this as the
    default response class for this endpoint.
    """
    return {
        "message": "Hello from ORJSONResponse",
        "timestamp": datetime.utcnow(),
        "data": [{"id": i, "value": f"item-{i}"} for i in range(100)],
    }


@app.get("/orjson-large-data", response_class=ORJSONResponse)
async def large_data():
    """
    ORJSONResponse excels with large data sets.
    """
    # Generate large dataset
    users = [
        {
            "id": i,
            "name": f"User {i}",
            "email": f"user{i}@example.com",
            "created_at": datetime.utcnow(),
            "uuid": str(uuid4()),
        }
        for i in range(1000)
    ]
    
    return {"users": users, "count": len(users)}


# Setting ORJSONResponse as the default for the entire app
app_with_orjson = FastAPI(default_response_class=ORJSONResponse)


@app_with_orjson.get("/fast-api")
async def fast_api():
    """
    All endpoints in this app use ORJSONResponse by default.
    """
    return {"message": "Fast JSON serialization!"}
```

#### ORJSONResponse Features

`orjson` provides additional features beyond speed:

```python
from fastapi import FastAPI
from fastapi.responses import ORJSONResponse
from datetime import datetime, timezone
from decimal import Decimal
from uuid import UUID, uuid4

app = FastAPI()


@app.get("/orjson-features", response_class=ORJSONResponse)
async def orjson_features():
    """
    ORJSONResponse handles types that standard json struggles with.
    """
    return {
        # Datetime is serialized as ISO 8601 by default
        "timestamp": datetime.now(timezone.utc),
        
        # UUID is serialized as string
        "uuid": uuid4(),
        
        # Decimal is serialized as number
        "price": Decimal("19.99"),
        
        # Bytes can be serialized (as base64 with option)
        "data": b"binary data",
        
        # Numpy arrays (if numpy is installed)
        # "array": numpy.array([1, 2, 3]),
    }
```

#### Custom Response Classes

Create custom response classes for specific needs:

```python
from fastapi import FastAPI
from fastapi.responses import JSONResponse
import orjson
from datetime import datetime
from typing import Any

app = FastAPI()


class PrettyJSONResponse(JSONResponse):
    """
    JSONResponse with pretty printing (indented).
    Useful for development and debugging.
    """
    
    def render(self, content: Any) -> bytes:
        return orjson.dumps(
            content,
            option=orjson.OPT_INDENT_2 | orjson.OPT_SERIALIZE_NUMPY,
        )


class CamelCaseJSONResponse(JSONResponse):
    """
    JSONResponse that converts snake_case to camelCase keys.
    """
    
    def render(self, content: Any) -> bytes:
        def to_camel_case(snake_str: str) -> str:
            components = snake_str.split("_")
            return components[0] + "".join(x.title() for x in components[1:])
        
        def transform_keys(obj: Any) -> Any:
            if isinstance(obj, dict):
                return {to_camel_case(k): transform_keys(v) for k, v in obj.items()}
            elif isinstance(obj, list):
                return [transform_keys(item) for item in obj]
            return obj
        
        transformed = transform_keys(content)
        return orjson.dumps(transformed)


@app.get("/pretty", response_class=PrettyJSONResponse)
async def pretty_json():
    """
    Returns nicely formatted JSON.
    """
    return {
        "message": "This is pretty printed JSON",
        "nested": {
            "data": [1, 2, 3],
            "timestamp": datetime.utcnow().isoformat(),
        },
    }


@app.get("/camel-case", response_class=CamelCaseJSONResponse)
async def camel_case_json():
    """
    Returns JSON with camelCase keys.
    """
    return {
        "user_id": 1,
        "user_name": "Alice",
        "created_at": "2024-01-15T12:00:00",
        "is_active": True,
    }
```

#### Performance Comparison Example

```python
import time
from fastapi import FastAPI
from fastapi.responses import JSONResponse, ORJSONResponse
from pydantic import BaseModel

app = FastAPI()


class LargeModel(BaseModel):
    id: int
    name: str
    value: float
    tags: list[str]
    metadata: dict[str, str]


def generate_large_data(count: int = 1000) -> list[dict]:
    """Generate test data for performance comparison."""
    return [
        {
            "id": i,
            "name": f"Item {i}",
            "value": i * 1.5,
            "tags": [f"tag-{j}" for j in range(10)],
            "metadata": {f"key-{k}": f"value-{k}" for k in range(5)},
        }
        for i in range(count)
    ]


@app.get("/benchmark/json")
async def benchmark_json():
    """
    Benchmark standard JSONResponse.
    """
    start = time.perf_counter()
    data = generate_large_data(1000)
    
    response = JSONResponse(content={"data": data})
    
    end = time.perf_counter()
    return {
        "library": "json (stdlib)",
        "time_ms": round((end - start) * 1000, 3),
        "items": len(data),
    }


@app.get("/benchmark/orjson", response_class=ORJSONResponse)
async def benchmark_orjson():
    """
    Benchmark ORJSONResponse.
    """
    start = time.perf_counter()
    data = generate_large_data(1000)
    
    # ORJSONResponse handles serialization
    end = time.perf_counter()
    return {
        "library": "orjson",
        "time_ms": round((end - start) * 1000, 3),
        "items": len(data),
        "data": data[:5],  # Only return first 5 items for display
    }
```

---

### 8.3 Streaming Responses: Sending Large Files or Real-Time Data

For large files, real-time updates, or when you want to send data as it's generated, streaming responses are essential. They reduce memory usage by not loading the entire response into memory at once.

#### Basic Streaming with `StreamingResponse`

```python
from fastapi import FastAPI
from fastapi.responses import StreamingResponse
import io

app = FastAPI()


@app.get("/stream-basic")
async def stream_basic():
    """
    Basic streaming response.
    Data is sent in chunks as generated.
    """
    
    async def generate():
        """Generator that yields data chunks."""
        for i in range(10):
            yield f"Chunk {i}\n"
    
    return StreamingResponse(
        generate(),
        media_type="text/plain",
    )


@app.get("/stream-json")
async def stream_json():
    """
    Stream JSON data line by line.
    Useful for large datasets.
    """
    
    async def generate():
        """Generate JSON lines."""
        import json
        for i in range(100):
            data = {"id": i, "name": f"Item {i}", "value": i * 10}
            yield json.dumps(data) + "\n"
    
    return StreamingResponse(
        generate(),
        media_type="application/x-ndjson",  # Newline-delimited JSON
    )
```

#### Streaming Large Files

```python
from pathlib import Path
from fastapi import FastAPI, HTTPException
from fastapi.responses import StreamingResponse, FileResponse

app = FastAPI()

FILES_DIR = Path("files")
FILES_DIR.mkdir(exist_ok=True)


@app.get("/download/{filename}")
async def download_large_file(filename: str):
    """
    Stream a large file to the client.
    Memory-efficient for large files.
    """
    file_path = FILES_DIR / filename
    
    if not file_path.exists():
        raise HTTPException(status_code=404, detail="File not found")
    
    def file_iterator():
        """Iterate over file in chunks."""
        with open(file_path, "rb") as f:
            while chunk := f.read(1024 * 1024):  # 1MB chunks
                yield chunk
    
    return StreamingResponse(
        file_iterator(),
        media_type="application/octet-stream",
        headers={
            "Content-Disposition": f'attachment; filename="{filename}"',
            "Content-Length": str(file_path.stat().st_size),
        },
    )


@app.get("/download-simple/{filename}")
async def download_simple(filename: str):
    """
    Simpler file download using FileResponse.
    FileResponse handles streaming internally.
    """
    file_path = FILES_DIR / filename
    
    if not file_path.exists():
        raise HTTPException(status_code=404, detail="File not found")
    
    return FileResponse(
        path=file_path,
        filename=filename,
        media_type="application/octet-stream",
    )
```

#### Asynchronous File Streaming

```python
import aiofiles
from pathlib import Path
from fastapi import FastAPI, HTTPException
from fastapi.responses import StreamingResponse

app = FastAPI()

FILES_DIR = Path("files")


@app.get("/download-async/{filename}")
async def download_async(filename: str):
    """
    Asynchronous file streaming using aiofiles.
    Non-blocking for better concurrency.
    """
    file_path = FILES_DIR / filename
    
    if not file_path.exists():
        raise HTTPException(status_code=404, detail="File not found")
    
    async def async_file_iterator():
        """Asynchronously iterate over file chunks."""
        async with aiofiles.open(file_path, "rb") as f:
            while chunk := await f.read(1024 * 1024):  # 1MB chunks
                yield chunk
    
    return StreamingResponse(
        async_file_iterator(),
        media_type="application/octet-stream",
        headers={
            "Content-Disposition": f'attachment; filename="{filename}"',
        },
    )
```

#### Server-Sent Events (SSE)

Server-Sent Events enable real-time server-to-client updates:

```python
import asyncio
import json
from datetime import datetime
from fastapi import FastAPI
from fastapi.responses import StreamingResponse

app = FastAPI()


@app.get("/events")
async def server_sent_events():
    """
    Server-Sent Events endpoint.
    Sends real-time updates to connected clients.
    """
    
    async def event_generator():
        """Generate SSE events."""
        count = 0
        while count < 10:  # Limit events for demo
            count += 1
            
            # Create event data
            data = {
                "id": count,
                "timestamp": datetime.utcnow().isoformat(),
                "message": f"Event {count}",
            }
            
            # SSE format: "data: {json}\n\n"
            yield f"data: {json.dumps(data)}\n\n"
            
            # Wait before next event
            await asyncio.sleep(1)
        
        # Send final event
        yield f"data: {json.dumps({'message': 'Stream complete'})}\n\n"
    
    return StreamingResponse(
        event_generator(),
        media_type="text/event-stream",
        headers={
            "Cache-Control": "no-cache",
            "Connection": "keep-alive",
            "X-Accel-Buffering": "no",  # Disable nginx buffering
        },
    )


@app.get("/events/named")
async def named_events():
    """
    SSE with named events and IDs.
    """
    
    async def event_generator():
        """Generate named SSE events."""
        events = [
            ("init", {"status": "connected"}),
            ("update", {"progress": 25}),
            ("update", {"progress": 50}),
            ("update", {"progress": 75}),
            ("complete", {"status": "done", "progress": 100}),
        ]
        
        for event_id, (event_name, data) in enumerate(events, 1):
            # Format: "id: {id}\nevent: {name}\ndata: {json}\n\n"
            yield f"id: {event_id}\n"
            yield f"event: {event_name}\n"
            yield f"data: {json.dumps(data)}\n\n"
            await asyncio.sleep(0.5)
    
    return StreamingResponse(
        event_generator(),
        media_type="text/event-stream",
    )
```

#### Real-Time Log Streaming

```python
import asyncio
from datetime import datetime
from fastapi import FastAPI
from fastapi.responses import StreamingResponse

app = FastAPI()


class LogManager:
    """Simple log manager for demonstration."""
    
    def __init__(self):
        self.subscribers = []
    
    def subscribe(self, queue: asyncio.Queue):
        self.subscribers.append(queue)
    
    def unsubscribe(self, queue: asyncio.Queue):
        self.subscribers.remove(queue)
    
    async def broadcast(self, message: str):
        """Broadcast log message to all subscribers."""
        for queue in self.subscribers:
            await queue.put(message)


log_manager = LogManager()


@app.get("/logs/stream")
async def stream_logs():
    """
    Stream application logs in real-time.
    """
    queue = asyncio.Queue()
    log_manager.subscribe(queue)
    
    async def log_generator():
        try:
            while True:
                # Wait for log message
                message = await asyncio.wait_for(queue.get(), timeout=30.0)
                yield f"data: {message}\n\n"
        except asyncio.TimeoutError:
            # Send keepalive
            yield f": keepalive\n\n"
    
    return StreamingResponse(
        log_generator(),
        media_type="text/event-stream",
    )


@app.post("/logs/add")
async def add_log(message: str):
    """
    Add a log message (broadcasts to all stream subscribers).
    """
    timestamp = datetime.utcnow().isoformat()
    formatted = f"[{timestamp}] {message}"
    await log_manager.broadcast(formatted)
    return {"status": "logged", "message": formatted}
```

#### CSV Streaming Export

```python
import io
import csv
from fastapi import FastAPI
from fastapi.responses import StreamingResponse

app = FastAPI()


@app.get("/export/csv")
async def export_csv():
    """
    Stream a large CSV file export.
    Memory-efficient for large datasets.
    """
    
    def generate_csv():
        """Generate CSV data in chunks."""
        output = io.StringIO()
        writer = csv.writer(output)
        
        # Write header
        writer.writerow(["ID", "Name", "Email", "Created At", "Status"])
        yield output.getvalue()
        output.seek(0)
        output.truncate(0)
        
        # Write data rows
        for i in range(1, 10001):  # 10,000 rows
            writer.writerow([
                i,
                f"User {i}",
                f"user{i}@example.com",
                f"2024-{(i % 12) + 1:02d}-{(i % 28) + 1:02d}",
                "active" if i % 2 == 0 else "inactive",
            ])
            yield output.getvalue()
            output.seek(0)
            output.truncate(0)
    
    return StreamingResponse(
        generate_csv(),
        media_type="text/csv",
        headers={
            "Content-Disposition": "attachment; filename=users_export.csv",
        },
    )
```

---

### 8.4 Custom Responses: HTML, XML, and Plain Text

FastAPI supports various response types beyond JSON. You can return HTML, XML, plain text, or create custom response formats.

#### HTML Response

```python
from fastapi import FastAPI
from fastapi.responses import HTMLResponse

app = FastAPI()


@app.get("/html", response_class=HTMLResponse)
async def get_html():
    """
    Return HTML content.
    """
    html_content = """
    <!DOCTYPE html>
    <html lang="en">
    <head>
        <meta charset="UTF-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0">
        <title>FastAPI HTML Response</title>
        <style>
            body {
                font-family: Arial, sans-serif;
                max-width: 800px;
                margin: 0 auto;
                padding: 20px;
                background-color: #f5f5f5;
            }
            .card {
                background: white;
                border-radius: 8px;
                padding: 20px;
                box-shadow: 0 2px 4px rgba(0,0,0,0.1);
            }
            h1 { color: #009688; }
        </style>
    </head>
    <body>
        <div class="card">
            <h1>Hello from FastAPI!</h1>
            <p>This is an HTML response from a FastAPI endpoint.</p>
            <ul>
                <li>Fast</li>
                <li>Easy</li>
                <li>Pythonic</li>
            </ul>
        </div>
    </body>
    </html>
    """
    return HTMLResponse(content=html_content)


@app.get("/page/{page_id}", response_class=HTMLResponse)
async def dynamic_html(page_id: int):
    """
    Generate dynamic HTML content.
    """
    html_content = f"""
    <!DOCTYPE html>
    <html>
    <head><title>Page {page_id}</title></head>
    <body>
        <h1>Page {page_id}</h1>
        <p>This is dynamically generated page {page_id}.</p>
        <nav>
            <a href="/page/1">Page 1</a> |
            <a href="/page/2">Page 2</a> |
            <a href="/page/3">Page 3</a>
        </nav>
    </body>
    </html>
    """
    return HTMLResponse(content=html_content)
```

#### XML Response

```python
from fastapi import FastAPI
from fastapi.responses import Response
from typing import Any
import xml.etree.ElementTree as ET

app = FastAPI()


class XMLResponse(Response):
    """
    Custom XML response class.
    """
    media_type = "application/xml"


def dict_to_xml(tag: str, data: Any) -> ET.Element:
    """
    Convert dictionary to XML element.
    """
    elem = ET.Element(tag)
    
    for key, val in data.items():
        child = ET.Element(key)
        if isinstance(val, dict):
            child.extend(dict_to_xml(key, val))
        elif isinstance(val, list):
            for item in val:
                child.append(dict_to_xml("item", item if isinstance(item, dict) else {"value": str(item)}))
        else:
            child.text = str(val)
        elem.append(child)
    
    return elem


@app.get("/xml", response_class=XMLResponse)
async def get_xml():
    """
    Return XML content.
    """
    xml_content = """<?xml version="1.0" encoding="UTF-8"?>
    <response>
        <status>success</status>
        <message>Hello from FastAPI XML!</message>
        <data>
            <item>
                <id>1</id>
                <name>Laptop</name>
                <price>999.99</price>
            </item>
            <item>
                <id>2</id>
                <name>Mouse</name>
                <price>29.99</price>
            </item>
        </data>
    </response>"""
    return XMLResponse(content=xml_content)


@app.get("/products/{product_id}/xml")
async def get_product_xml(product_id: int):
    """
    Return a single product as XML.
    """
    # Simulated product data
    product = {
        "id": product_id,
        "name": f"Product {product_id}",
        "price": 99.99,
        "description": "A great product",
        "stock": 100,
    }
    
    # Convert to XML
    root = dict_to_xml("product", product)
    xml_string = ET.tostring(root, encoding="unicode")
    xml_with_declaration = f'<?xml version="1.0" encoding="UTF-8"?>\n{xml_string}'
    
    return XMLResponse(content=xml_with_declaration)
```

#### Plain Text Response

```python
from fastapi import FastAPI
from fastapi.responses import PlainTextResponse

app = FastAPI()


@app.get("/text", response_class=PlainTextResponse)
async def get_text():
    """
    Return plain text content.
    """
    return "Hello from FastAPI!\nThis is a plain text response."


@app.get("/logs/{log_id}", response_class=PlainTextResponse)
async def get_log(log_id: int):
    """
    Return log file content as plain text.
    """
    # Simulated log content
    log_content = f"""
[2024-01-15 10:30:00] INFO: Application started
[2024-01-15 10:30:01] INFO: Log {log_id} requested
[2024-01-15 10:30:02] DEBUG: Processing request
[2024-01-15 10:30:03] INFO: Request completed
"""
    return log_content.strip()


@app.get("/data.txt")
async def get_data_txt():
    """
    Return data as plain text file download.
    """
    from fastapi.responses import Response
    
    content = "\n".join([f"Item {i}: Value {i * 10}" for i in range(1, 101)])
    
    return Response(
        content=content,
        media_type="text/plain",
        headers={
            "Content-Disposition": "attachment; filename=data.txt",
        },
    )
```

#### RSS/Atom Feed Response

```python
from datetime import datetime
from fastapi import FastAPI
from fastapi.responses import Response

app = FastAPI()


@app.get("/rss", response_class=Response)
async def get_rss():
    """
    Generate an RSS feed.
    """
    # Simulated feed items
    items = [
        {
            "title": "FastAPI 1.0 Released",
            "link": "https://example.com/news/fastapi-1-0",
            "description": "FastAPI version 1.0 has been released with exciting new features.",
            "pub_date": "Mon, 15 Jan 2024 10:00:00 GMT",
        },
        {
            "title": "Best Practices for API Design",
            "link": "https://example.com/news/api-best-practices",
            "description": "Learn the best practices for designing REST APIs.",
            "pub_date": "Sun, 14 Jan 2024 15:30:00 GMT",
        },
        {
            "title": "Understanding Async in Python",
            "link": "https://example.com/news/async-python",
            "description": "A deep dive into asynchronous programming in Python.",
            "pub_date": "Sat, 13 Jan 2024 09:00:00 GMT",
        },
    ]
    
    # Generate RSS XML
    rss_content = f"""<?xml version="1.0" encoding="UTF-8"?>
<rss version="2.0" xmlns:atom="http://www.w3.org/2005/Atom">
<channel>
    <title>Example News Feed</title>
    <link>https://example.com</link>
    <description>Latest news and updates</description>
    <language>en-us</language>
    <lastBuildDate>{datetime.utcnow().strftime("%a, %d %b %Y %H:%M:%S GMT")}</lastBuildDate>
    <atom:link href="https://example.com/rss" rel="self" type="application/rss+xml"/>
    {"".join(f"""
    <item>
        <title>{item['title']}</title>
        <link>{item['link']}</link>
        <description>{item['description']}</description>
        <pubDate>{item['pub_date']}</pubDate>
        <guid>{item['link']}</guid>
    </item>""" for item in items)}
</channel>
</rss>"""
    
    return Response(
        content=rss_content,
        media_type="application/rss+xml",
        headers={
            "Content-Type": "application/rss+xml; charset=utf-8",
        },
    )
```

#### Custom Binary Response

```python
from io import BytesIO
from fastapi import FastAPI
from fastapi.responses import Response
from PIL import Image, ImageDraw, ImageFont

app = FastAPI()


@app.get("/image/dynamic")
async def generate_image(width: int = 400, height: int = 200, text: str = "Hello!"):
    """
    Generate a dynamic image.
    Requires Pillow: pip install Pillow
    """
    # Create image
    img = Image.new("RGB", (width, height), color=(73, 109, 137))
    
    # Draw on image
    draw = ImageDraw.Draw(img)
    
    # Add text (uses default font)
    draw.text((width // 2, height // 2), text, fill="white", anchor="mm")
    
    # Add a border
    draw.rectangle([0, 0, width - 1, height - 1], outline=(255, 255, 255), width=2)
    
    # Save to bytes
    img_bytes = BytesIO()
    img.save(img_bytes, format="PNG")
    img_bytes.seek(0)
    
    return Response(
        content=img_bytes.getvalue(),
        media_type="image/png",
        headers={
            "Cache-Control": "public, max-age=3600",
        },
    )


@app.get("/download/binary")
async def download_binary():
    """
    Generate and download a binary file.
    """
    # Generate some binary data
    binary_data = bytes(range(256)) * 4  # 1KB of binary data
    
    return Response(
        content=binary_data,
        media_type="application/octet-stream",
        headers={
            "Content-Disposition": "attachment; filename=data.bin",
            "Content-Length": str(len(binary_data)),
        },
    )
```

#### Content Negotiation

```python
from fastapi import FastAPI, Request, HTTPException
from fastapi.responses import JSONResponse, HTMLResponse, PlainTextResponse, XMLResponse

app = FastAPI()


@app.get("/resource/{resource_id}")
async def get_resource(request: Request, resource_id: int):
    """
    Content negotiation: Return different formats based on Accept header.
    """
    # Resource data
    data = {
        "id": resource_id,
        "name": f"Resource {resource_id}",
        "description": "This is a sample resource",
        "created_at": "2024-01-15T12:00:00Z",
    }
    
    # Get Accept header
    accept = request.headers.get("accept", "application/json")
    
    # Determine response format
    if "application/json" in accept or accept == "*/*":
        return JSONResponse(content=data)
    
    elif "text/html" in accept:
        html = f"""
        <!DOCTYPE html>
        <html>
        <body>
            <h1>{data['name']}</h1>
            <p>ID: {data['id']}</p>
            <p>{data['description']}</p>
            <p>Created: {data['created_at']}</p>
        </body>
        </html>
        """
        return HTMLResponse(content=html)
    
    elif "text/plain" in accept:
        text = f"""
Resource ID: {data['id']}
Name: {data['name']}
Description: {data['description']}
Created: {data['created_at']}
        """.strip()
        return PlainTextResponse(content=text)
    
    elif "application/xml" in accept:
        xml = f"""<?xml version="1.0"?>
<resource>
    <id>{data['id']}</id>
    <name>{data['name']}</name>
    <description>{data['description']}</description>
    <created_at>{data['created_at']}</created_at>
</resource>"""
        return XMLResponse(content=xml)
    
    else:
        raise HTTPException(
            status_code=406,
            detail="Acceptable formats: application/json, text/html, text/plain, application/xml",
        )
```

#### Complete Multi-Format API Example

```python
from typing import Any
from fastapi import FastAPI, Request, Path, Query
from fastapi.responses import (
    JSONResponse,
    HTMLResponse,
    PlainTextResponse,
    Response,
    StreamingResponse,
)
from pydantic import BaseModel
import csv
import io

app = FastAPI(title="Multi-Format API")


# Models
class Product(BaseModel):
    id: int
    name: str
    price: float
    category: str
    in_stock: bool


class ProductList(BaseModel):
    products: list[Product]
    total: int


# Sample data
products_db = [
    Product(id=1, name="Laptop", price=999.99, category="Electronics", in_stock=True),
    Product(id=2, name="Mouse", price=29.99, category="Electronics", in_stock=True),
    Product(id=3, name="Desk", price=199.99, category="Furniture", in_stock=False),
    Product(id=4, name="Chair", price=149.99, category="Furniture", in_stock=True),
    Product(id=5, name="Monitor", price=399.99, category="Electronics", in_stock=True),
]


# Response formats
def format_json(data: Any) -> Response:
    return JSONResponse(content=data)


def format_html(data: Any) -> Response:
    if isinstance(data, dict) and "products" in data:
        products = data["products"]
        rows = "".join(
            f"<tr><td>{p['id']}</td><td>{p['name']}</td><td>${p['price']}</td>"
            f"<td>{p['category']}</td><td>{'✓' if p['in_stock'] else '✗'}</td></tr>"
            for p in products
        )
        html = f"""
        <!DOCTYPE html>
        <html>
        <head><title>Products</title>
        <style>
            table {{ border-collapse: collapse; width: 100%; }}
            th, td {{ border: 1px solid #ddd; padding: 8px; text-align: left; }}
            th {{ background-color: #4CAF50; color: white; }}
            tr:nth-child(even) {{ background-color: #f2f2f2; }}
        </style>
        </head>
        <body>
            <h1>Products ({data['total']} items)</h1>
            <table>
                <tr><th>ID</th><th>Name</th><th>Price</th><th>Category</th><th>In Stock</th></tr>
                {rows}
            </table>
        </body>
        </html>"""
    else:
        html = f"<pre>{data}</pre>"
    return HTMLResponse(content=html)


def format_text(data: Any) -> Response:
    if isinstance(data, dict) and "products" in data:
        lines = ["ID | Name | Price | Category | In Stock"]
        lines.append("-" * 50)
        for p in data["products"]:
            stock = "Yes" if p["in_stock"] else "No"
            lines.append(f"{p['id']} | {p['name']} | ${p['price']} | {p['category']} | {stock}")
        text = "\n".join(lines)
    else:
        text = str(data)
    return PlainTextResponse(content=text)


def format_csv(data: Any) -> Response:
    def generate():
        output = io.StringIO()
        writer = csv.writer(output)
        writer.writerow(["ID", "Name", "Price", "Category", "In Stock"])
        yield output.getvalue()
        output.seek(0)
        output.truncate(0)
        
        for p in data["products"]:
            writer.writerow([p["id"], p["name"], p["price"], p["category"], p["in_stock"]])
            yield output.getvalue()
            output.seek(0)
            output.truncate(0)
    
    return StreamingResponse(
        generate(),
        media_type="text/csv",
        headers={"Content-Disposition": "attachment; filename=products.csv"},
    )


@app.get("/products")
async def list_products(
    request: Request,
    category: str | None = Query(default=None),
    format: str = Query(default="json", regex="^(json|html|text|csv)$"),
):
    """
    List products in various formats.
    
    Query parameters:
    - category: Filter by category
    - format: Response format (json, html, text, csv)
    """
    # Filter products
    filtered = products_db
    if category:
        filtered = [p for p in products_db if p.category.lower() == category.lower()]
    
    data = {
        "products": [p.model_dump() for p in filtered],
        "total": len(filtered),
    }
    
    # Return requested format
    if format == "json":
        return format_json(data)
    elif format == "html":
        return format_html(data)
    elif format == "text":
        return format_text(data)
    elif format == "csv":
        return format_csv(data)
    
    return format_json(data)


@app.get("/products/{product_id}")
async def get_product(
    product_id: int = Path(ge=1),
    format: str = Query(default="json", regex="^(json|html|text)$"),
):
    """
    Get a single product in various formats.
    """
    product = next((p for p in products_db if p.id == product_id), None)
    
    if not product:
        from fastapi import HTTPException
        raise HTTPException(status_code=404, detail="Product not found")
    
    data = product.model_dump()
    
    if format == "json":
        return format_json(data)
    elif format == "html":
        return format_html(data)
    elif format == "text":
        return format_text(data)
    
    return format_json(data)
```

---

### Summary

In this chapter, you've mastered response handling in FastAPI:

1. **Status Codes**: Using appropriate HTTP status codes for different scenarios and setting them via `status_code`, `Response` object, or response classes.

2. **JSON Response Performance**: Using `JSONResponse` and `ORJSONResponse` for optimized JSON serialization, and creating custom response classes for specific needs.

3. **Streaming Responses**: Implementing `StreamingResponse` for large files, real-time data, and Server-Sent Events.

4. **Custom Responses**: Returning HTML, XML, plain text, RSS feeds, and implementing content negotiation.

---

### Exercises

1. **Status Code Practice**: Create an API endpoint that:
   - Returns 201 with Location header on resource creation
   - Returns 202 for async processing
   - Returns 204 for successful deletion
   - Returns 409 for conflict
   - Returns 304 for conditional GET with ETag

2. **Performance Optimization**: Build endpoints that:
   - Use ORJSONResponse for large datasets (10,000+ items)
   - Compare response times between JSONResponse and ORJSONResponse
   - Create a custom response class with camelCase transformation

3. **Streaming Implementation**: Create endpoints that:
   - Stream a large CSV file (100,000 rows)
   - Implement Server-Sent Events for real-time notifications
   - Stream log files from disk with pagination

4. **Multi-Format API**: Build a complete API that:
   - Supports JSON, XML, HTML, and CSV output formats
   - Uses content negotiation based on Accept header
   - Provides a query parameter override for format selection
   - Includes proper caching headers for each format

---

### What's Next?

**Chapter 9: Project Architecture** will guide you through structuring larger FastAPI applications:
- Modularization: Breaking the app into multiple files
- Using `APIRouter` to create mini-applications
- The "Service-Repository" pattern for clean separation of concerns
- Configuration management and centralizing settings



<div style='width:100%; display:flex; justify-content:space-between; align-items:center; margin: 1em 0;'>
  <a href='7. request_handling_and_context.ipynb' style='font-weight:bold; font-size:1.05em;'>&larr; Previous</a>
  <a href='../TOC.md' style='font-weight:bold; font-size:1.05em; text-align:center;'>Table of Contents</a>
  <a href='../4. structuring_large_applications/9. project_architecture.ipynb' style='font-weight:bold; font-size:1.05em;'>Next &rarr;</a>
</div>
